In [1]:
%set_env PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync

from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PDFPlumberLoader

pypdf_loader = PyPDFLoader("data/MGRP/MGRP Unmarked 00 - Prologue.pdf")
pypdf_pages = pypdf_loader.load_and_split()

loader = PDFPlumberLoader("data/MGRP/MGRP Unmarked 00 - Prologue.pdf")
pages = loader.load_and_split()

print(pypdf_pages[0])
print(pages[0])

env: PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync


KeyboardInterrupt: 

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

db = Chroma.from_documents(pages, OpenAIEmbeddings())
retriever = db.as_retriever()
results = retriever.get_relevant_documents("What is the real world name of Snow White?")

print(results[0])

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")
total_token_count = 0

embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")


def tiktoken_length(text):
    global total_token_count
    token_count = encoding.encode(text)
    if len(token_count) == 0:
        return 0
    else:
        total_token_count += token_count[0]
        return token_count[0]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=tiktoken_length,
    add_start_index=True,
)
texts = text_splitter.create_documents(pages[0].page_content)
print(total_token_count)
print(texts[0])
print(texts[1])
db = Chroma.from_documents(pages, embeddings)
retriever = db.as_retriever()
results = retriever.get_relevant_documents("What is the real world name of Snow White?")

print("\n")
print(results[0])